# 策略说明

    开仓条件：
        1. 日线沿5日均线上线,收盘不破5日线
        2. 5日内存在涨幅大于4%的情形
        3. 当日收阴
        4. 按收盘价与均线之间的gap排序，买入较接近的
    平仓条件：
        1. 在日线高位的情况下，30分钟RSI顶背离

In [1]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsi30, rsiday
from alpha.features.volume import *
await init_notebook()

In [37]:
def day_features(code, name, bars, results, frame_type):
    close = bars["close"]
    c0 = close[-1]
    ma = moving_average(close, 5)
    
    n = 20
    flag, pos, length = find_runs(close[-n:] >= ma[-n:])
    if not flag[-1]:
        return
    
    msr = length[-1]
    
    roc = max(close[1:]/close[:-1]-1)
    gap = c0/ma[-1] - 1
    profit = c0/min(close) - 1
        
    v1, v0 = top_volume_direction(bars, 10)
    nv = moving_net_volume(bars)

    results.append((name, code, gap, profit, msr, roc, v1, v0, nv[-1]))
    
    
def m30_features(code, name, bars, results, frame_type):
    close = bars["close"]

    rsi = relative_strength_index(close, period=6)
    prsi = np.array([rsi30.get_proba(code, r) for r in rsi[-3:]])

    # RSI 超跌
    if np.any(prsi < 0.1) and close[-1] > close[-3]:
        results.append((code, name, "rsi"))
        return

    # 5线顺上
    ma = moving_average(close, 5)
    n = 3
    if np.count_nonzero(close[-n:] >= ma[-n:]) >= n:
        results.append((code, name, "均线顺上"))
        return

    # 5日均线反转
    ma = ma[-10:]
    (a, b, c), pmae = polyfit(ma/ma[0])
    vx = round((-1 * b) / (2 * a)) 
    if b < -1e-3 and (a > 5e-4 and vx < 8) and pmae < 1e-3:
        results.append((code, name, "圆弧底"))
        return

    results.append((code, name, "无信号"))

In [40]:
async def strategy(tm=None):
    tm = tm or arrow.now()
    results = await scan(day_features, 30, nstocks=-1, silent=True, tm=tm)
#     codes = [r[0] for r in results]
    
#     day_results = {
#         r[0]: (r[2:]) for r in results
#     }
    
    #m_results = await scan(m30_features, 60, FrameType.MIN30, codes=codes)
    
#     print("min results", len(m_results))
#     results = [
#         (*r, *day_results[r[0]]) for r in m_results
#     ]

    #names = MaLineFeatures.feature_names(60, [5, 10, 20, 30])
    df = pd.DataFrame(results, columns = ["code", "name", "gap","profit","msr", "roc", "v1", "v0", "nv"])
    return df

In [41]:
df = await strategy()

2021-12-06
progress: 500/4018, results: 204, elapsed: 2, ETA: 14
progress: 1000/4018, results: 325, elapsed: 3, ETA: 9
progress: 1500/4018, results: 439, elapsed: 3, ETA: 5
progress: 2000/4018, results: 519, elapsed: 4, ETA: 4
progress: 2500/4018, results: 604, elapsed: 10, ETA: 6
progress: 3000/4018, results: 817, elapsed: 10, ETA: 3
progress: 3500/4018, results: 1052, elapsed: 11, ETA: 1
progress: 4000/4018, results: 1146, elapsed: 15, ETA: 0


In [57]:
pd.options.display.max_rows=120
df[(df.profit<0.2)&(df.gap<4e-2)&(df.roc>4e-2)&(df.v1>5)].sort_values("nv")

,code,name,gap,profit,msr,roc,v1,v0,nv
369,中装建设,002822.XSHE,0.015340,0.116466,5,0.100569,10.325290,-0.819990,-0.102293
800,凤凰股份,600716.XSHG,0.003557,0.034032,2,0.045802,7.148020,-0.563510,-0.020695
961,通用股份,601500.XSHG,0.013946,0.052838,5,0.051625,7.664940,-0.614969,5.032209
945,广州港,601228.XSHG,0.011029,0.061093,5,0.090909,15.147514,-0.562718,5.858512
197,南京港,002040.XSHE,0.005008,0.054291,4,0.040404,7.068855,-0.587385,6.319247
913,一拖股份,601038.XSHG,0.016677,0.095631,5,0.055771,5.259598,-0.586165,6.795920
21,盐田港,000088.XSHE,0.006878,0.039448,4,0.046422,9.359462,-0.451316,6.811880
434,银之杰,300085.XSHE,0.007432,0.125288,1,0.077548,5.838502,-0.550280,6.955748
643,泉阳泉,600189.XSHG,0.015455,0.161602,4,0.100267,6.635423,-0.366841,8.054861
255,桂林三金,002275.XSHE,0.013015,0.124408,1,0.044063,5.729444,-0.280081,15.326846


In [68]:
df[(df.nv>10)&(df.msr>3)&(df.roc>3e-2)&(df.profit<0.3)].sort_values("gap")

,code,name,gap,profit,msr,roc,v1,v0,nv
1058,济民医疗,603222.XSHG,0.000278,0.054212,11,0.040087,2.431343,-0.663879,11.241885
620,波导股份,600130.XSHG,0.006413,0.214286,5,0.100000,-3.415943,0.788290,15.777818
678,曙光股份,600303.XSHG,0.008403,0.186046,5,0.046575,2.596011,-0.502019,10.322516
720,中远海特,600428.XSHG,0.010757,0.126033,6,0.073022,3.066497,-0.526417,11.247335
499,盛讯达,300518.XSHE,0.011448,0.129916,4,0.068829,2.761967,-0.620846,10.005584
276,永安药业,002365.XSHE,0.015457,0.195853,12,0.040354,3.517965,-0.701196,22.813654
61,甘化科工,000576.XSHE,0.020574,0.286381,5,0.100422,3.892351,-0.588078,13.814375
821,安徽合力,600761.XSHG,0.021344,0.250726,5,0.061564,-1.691691,0.000000,16.872155
304,山东墨龙,002490.XSHE,0.021574,0.201493,4,0.099515,6.195910,-0.680855,11.073086
625,长春一东,600148.XSHG,0.022316,0.200851,6,0.062691,5.851079,-0.493948,27.621259


In [ ]:
## 持仓4天强平

In [ ]:
z05 = Z05(holding_days=4)
summary = await z05.backtest("2021-10-21", "2021-10-21")

In [ ]:
summary

In [ ]:
z05.trades

# 30日回测

持仓最长10天

In [ ]:
from itertools import product
import random
import logging

rootLogger = logging.getLogger()
rootLogger.setLevel(logging.WARNING)

rsi = [70, 75, 80, 85]
prsi = [0.7, 0.75, 0.8, 0.85]
rsi3 = [80, 85, 90, 95]
stop_loss = [-0.05, -0.1, -0.15]
msr = [0.6, 0.7, 0.8, 0.9, 1]
bcr = [0.6, 0.7, 0.8, 0.9, 1]
d1 = np.arange(1,4)/100

all_perms = list(product(*(rsi, prsi, rsi3, stop_loss, msr, bcr, d1)))

results = []

for params in random.sample(all_perms, int(0.1 * len(all_perms))):
    rsi, prsi, rsi3, stop_loss, msr, bcr, d1 = params
    z05_30 = Z05(holding_days=10, rsi=rsi, prsi=prsi, rsi3=rsi3, stop_loss=stop_loss,msr=msr,bcr=bcr,d1=d1)

    summary = await z05_30.backtest("2021-09-01", "2021-09-03")
    summary.update({
        "rsi": rsi,
        "prsi": prsi,
        "rsi3": rsi3,
        "stop_loss": stop_loss,
        "msr": msr,
        "bcr": bcr,
        "d1": d1
    })
    results.append(summary)
df = pd.DataFrame(results)

In [ ]:
columns = ["name", "code", "gains", "buy_at", "sell_at", "duration",  "buy", "sell", "type", "params"]
df = pd.DataFrame(z05_30.orders, columns=columns)

pd.options.display.max_rows = 999
df